In [1]:
%reload_ext autoreload
%autoreload 1
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load
from utils.loader import PairSetMNIST
from torch.utils.data import Dataset, DataLoader
from utils.metrics import accuracy, compute_nb_errors

In [2]:
class LeNet_aux_sequential(nn.Module):
    
    """
    Weight sharing + Auxiliary loss
    
    """
    def __init__(self):
        super(LeNet_aux_sequential, self).__init__()
        # convolutional weights for digit reocgnition shared for each image
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        
        # weights for binary classification 
        self.fc3 = nn.Linear(20, 60)
        self.fc4 = nn.Linear(60, 90)
        self.fc5 = nn.Linear(90, 2)
        
    def forward(self, input_):    
        
        # split the 2-channel input into two 14*14 images
        x = input_[:, 0, :, :].view(-1, 1, 14, 14)
        y = input_[:, 1, :, :].view(-1, 1, 14, 14)
        
        # forward pass for the first image 
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        
        # forward pass for the second image 
        y = F.relu(F.max_pool2d(self.conv1(y), kernel_size=2, stride=2))
        y = F.relu(F.max_pool2d(self.conv2(y), kernel_size=2, stride=2))
        y = F.relu(self.fc1(y.view(-1, 256)))
        y = self.fc2(y)
        
        # concatenate layers  
        z = torch.cat([x, y], 1)
        
        z = F.relu(self.fc3(z))
        z = F.relu(self.fc4(z))
        z = self.fc5(z)
        
        return x, y, z

In [3]:
#train for one epoch
def train_aux (model, train_data, mini_batch_size=100, optimizer = optim.SGD,
                criterion = nn.CrossEntropyLoss(), n_epochs=50, eta=1e-1, lambda_l2 = 0, alpha=0.5, beta=0.5):
    
    """
    Train network with auxiliary loss + weight sharing
    
    """
    model.train()
    optimizer = optimizer(model.parameters(), lr = eta)
    
    for e in range(n_epochs):
        epoch_loss = 0
        # create data loader
        train_loader = DataLoader(train_data, batch_size=mini_batch_size, shuffle=True)
        
        for i, data in enumerate(train_loader, 0):
            
            input_, target_, classes_ = data
            class_1, class_2, out = model(input_)
            aux_loss1 = criterion(class_1, classes_[:,0])
            aux_loss2 = criterion(class_2, classes_[:,1])
            out_loss  = criterion(out, target_)
            net_loss = (alpha * (out_loss) + beta * (aux_loss1 + aux_loss2) )
            epoch_loss += net_loss
            
            if lambda_l2 != 0:
                for p in model.parameters():
                    epoch_loss += lambda_l2 * p.pow(2).sum() # add an l2 penalty term to the loss 
            
            optimizer.zero_grad()
            net_loss.backward()
            optimizer.step()
            
        print('Train Epoch: {}  | Loss {:.6f}'.format(
                e, epoch_loss.item()))
        
# test on a given test dataset
def test_aux(model, test_data, mini_batch_size=100, criterion = nn.CrossEntropyLoss()):
    
    """
    Test function to calculate prediction accuracy of a cnn with auxiliary loss
    
    """
    test_loader = DataLoader(test_data, batch_size=mini_batch_size, shuffle=True)
    model.eval()
    test_loss = 0
    nb_errors = 0
    
    with torch.no_grad():
        
        for i, data in enumerate(test_loader, 0):
            input_, target_, classes_ = data
            _, _, output = model(input_) 
            batch_loss = criterion(output, target_)
            test_loss += batch_loss     
            nb_errors += compute_nb_errors(output, target_)
            
        acc = 100*(nb_errors/len(test_loader))
      
                    
        print('\nSet | Loss: {:.4f} | Accuracy: {:.2f}%\n'.format(test_loss.item(), acc))
        
        return test_loss, acc

# run a model for 10 rounds of train/test and compute results

def test_trial(n_trial, n_epochs_):
    
    train_data = PairSetMNIST(train=True)
    test_data  = PairSetMNIST(test=True)

    accuracies_test = torch.empty(n_trial, dtype = float)
    accuracies_train = torch.empty(n_trial, dtype = float)
    losses_train = torch.empty(n_trial, dtype = float)
    losses_test = torch.empty(n_trial, dtype = float)

    for n in range(n_trial):

        model = LeNet_aux_sequential()
        model.train(True)
        train_aux (model, train_data, n_epochs=n_epochs_ )
        test_loss, acc_test = test_aux(model, test_data)
        train_loss, acc_train = test_aux(model, train_data)

        accuracies_test[n] = acc_test
        accuracies_train[n] = acc_train
        losses_test[n] = test_loss
        losses_train[n] = train_loss
        
        
    return accuracies_train, losses_train, accuracies_test, losses_test

In [4]:
###############################
######## Auxiliary Loss #######
###############################

#model = LeNet_aux_sequential()
#train_aux(model, train_data)
#test_loss, acc = test_aux(model,test_data)
accuracies_train, losses_train, accuracies_test, losses_test = test_trial(n_trial=5, n_epochs_=3)


Train Epoch: 0  | Loss 25.795433
Train Epoch: 1  | Loss 23.362532
Train Epoch: 2  | Loss 19.193148

Set | Loss: 6.9607 | Accuracy: 4730.00%


Set | Loss: 6.8350 | Accuracy: 4490.00%

Train Epoch: 0  | Loss 26.018250
Train Epoch: 1  | Loss 24.082714
Train Epoch: 2  | Loss 19.878902

Set | Loss: 6.9190 | Accuracy: 4760.00%


Set | Loss: 6.8686 | Accuracy: 4490.00%

Train Epoch: 0  | Loss 25.994595
Train Epoch: 1  | Loss 24.088398
Train Epoch: 2  | Loss 21.275850

Set | Loss: 6.8696 | Accuracy: 4750.00%


Set | Loss: 6.8181 | Accuracy: 4490.00%

Train Epoch: 0  | Loss 25.696764
Train Epoch: 1  | Loss 23.007717
Train Epoch: 2  | Loss 19.699543

Set | Loss: 6.7992 | Accuracy: 4630.00%


Set | Loss: 6.7565 | Accuracy: 4400.00%

Train Epoch: 0  | Loss 26.229111
Train Epoch: 1  | Loss 25.138542
Train Epoch: 2  | Loss 22.811392

Set | Loss: 6.9277 | Accuracy: 4760.00%


Set | Loss: 6.8706 | Accuracy: 4490.00%



In [15]:
test_data = PairSetMNIST(test=True)
test_loader = DataLoader(test_data, batch_size=100, shuffle=True)
test_loader[0]


TypeError: 'DataLoader' object is not subscriptable

In [7]:
accuracies_train

tensor([400., 400., 400., 400., 400.], dtype=torch.float64)

In [15]:
type(accuracies_test)

torch.Tensor

In [5]:
x = torch.tensor([1, 2, 3, 4, 5 ,7, 8, 8])
y = torch.tensor([3, 5, 3, 8, 4, 7, 8, 5])


In [6]:
(x == y).sum().float()/ ( x.shape[0])

tensor(0.3750)